# EM + Atlas 

In [1]:
from os import listdir
from os.path import isdir, join
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from numpy.linalg import inv, det, norm
from math import sqrt, pi
from functools import partial
#from scipy.spatial.distance import dice

/home/okozyn/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# numeric labels for brain tissue types
CSF_label = 1
GM_label = 3
WM_label = 2

In [3]:
def normalize(target_array):
    """
    Normalize array values from 0 to 1.
    """
    target_array -= target_array.min()
    normalized_array = np.divide(target_array, target_array.max())
    return normalized_array


def show_slice(img, slice_no):
    """
        Inputs: img (nibabel): image name
                slice_no (np slice): np.s_[:, :, 30]
    """
    data = img.get_fdata()
    plt.figure()
    plt.imshow(data[slice_no].T, cmap='gray')
    plt.show()
    

def show_slice_data(data, slice_no):
    """
    Display slice of a given array.
    """
    plt.imshow(data[slice_no], cmap = "gray")
    plt.show()

    
def read_im(image_path):
    """
    Read nii file from image_path (str).
    """
    nii_img = nib.load(image_path)
    nii_data = nii_img.get_data()
    
    return nii_data, nii_img


def calc_dice(segmented_images, groundtruth_images):
    """
    Calcualte dice similarity coefficient between two volumes.
    """
    segData = segmented_images + groundtruth_images
    TP_value = np.amax(segmented_images) + np.amax(groundtruth_images)
    
    # found a true positive: segmentation result and groundtruth match(both are positive)
    TP = (segData == TP_value).sum()
    segData_FP = 2. * segmented_images + groundtruth_images
    segData_FN = segmented_images + 2. * groundtruth_images
    
    # found a false positive: segmentation result and groundtruth mismatch
    FP = (segData_FP == 2 * np.amax(segmented_images)).sum() 
    
    # found a false negative: segmentation result and groundtruth mismatch
    FN = (segData_FN == 2 * np.amax(groundtruth_images)).sum() 
    
    return 2*TP/(2*TP+FP+FN)  # according to the definition of DICE similarity score


def dice_similarity(segmented_img, groundtruth_img):
    """
    Extract binary label images for regions
        Inputs: segmented_img (nibabel): segmented labels nii file
                groundtruth_img (nibabel): groundtruth labels nii file
        Returns: DICE_index (float): Dice similarity score between the two images (nii files)        
    """
    
    segmented_data = segmented_img.get_data().copy()
    groundtruth_data = groundtruth_img.get_data().copy()
    seg_CSF = (segmented_data == CSF_label) * 1
    gt_CSF = (groundtruth_data == CSF_label) * 1
    seg_GM = (segmented_data == GM_label) * 1
    gt_GM = (groundtruth_data == GM_label) * 1
    seg_WM = (segmented_data == WM_label) * 1
    gt_WM = (groundtruth_data == WM_label) * 1
    
    dice_CSF = calc_dice(seg_CSF, gt_CSF)
    dice_GM = calc_dice(seg_GM, gt_GM)
    dice_WM = calc_dice(seg_WM, gt_WM)
    
    
    return dice_CSF, dice_GM, dice_WM


def apply_mask(target_data, gt_data):
    """
    Create mask using groundtruth image and apply it.
    
    Inputs: 
        gt_img: groundtuth mask
        target_img: raw data, apply mask to it
    
    Returns: 
        masked_img: target image with mask applied (background removed)
    """
    
    
    # Create mask: Select pixels higher than 0 in gt and set to 1
    gt_data[gt_data > 0] = 1
    
    # Apply mask
    target_data = np.multiply(target_data, gt_data)
    
    
    return target_data


def seg_data_to_nii(original_im, y_pred, features_nonzero_row_indicies):
    """
        Inputs: original_im (nibabel): original image nii file
                y_pred (np array): labels for all non-zero points
                features_nonzero_row_indicies (np array): indicies of non-zero points,
                                                          same length as y_pred
        Returns: segment_nii (nibabel): segmented labels nii file        
    """
    original_img_shape = original_im.get_data().shape
    original_img_len = original_img_shape[0] * original_img_shape[1] * original_img_shape[2]
    segment_im = np.zeros(original_img_len)
    labels = np.copy(y_pred) + 1
    segment_im[features_nonzero_row_indicies] = labels
    segment_im = np.reshape(segment_im, original_im.shape)
    segment_nii = nib.Nifti1Image(segment_im, original_im.affine, original_im.header)
    
    return segment_nii


def integrate_atlas_nii(original_im, y_pred, features_nonzero, features_nonzero_row_indicies, weights, csf_atlas, 
                           gm_atlas, wm_atlas):
    """
    Transforms segmenation result to nii file, puts correct labels in place.
    The segmentation labels should be: 1) CSF (darkest) 2) GM (middle) 3) WM (light)
    
    Inputs: 
    original_im (nibabel): original image nii file
    y_pred (np array): labels for all non-zero points
    features_nonzero (np array): feature vector of only non-zero intensities
    features_nonzero_row_indicies (np array): indicies of non-zero points,
                                              same length as y_pred

    Returns:
    segment_nii (nibabel): segmented labels nii file        
    """
    
    # Create image with all 3 classes and random labels
    y_pred = y_pred + 1
    original_img_shape = original_im.get_data().shape
    original_img_len = original_img_shape[0] * original_img_shape[1] * original_img_shape[2]
    
    segment_im = np.zeros(original_img_len)
    segment_im[features_nonzero_row_indicies] = y_pred
    segment_im = np.reshape(segment_im, original_im.shape)
    
    temp_class1_im = np.zeros_like(segment_im)
    temp_class2_im = np.zeros_like(segment_im)
    temp_class3_im = np.zeros_like(segment_im)
    
    #Assign class1 to 1
    temp_class1_im[segment_im == 1] = 1
    #Assign class2 to 2
    temp_class2_im[segment_im == 2] = 1
    #Assign class3 to 1
    temp_class3_im[segment_im == 3] = 1
    
    # Compute DICE between each class to determine which class it belongs to
    dice1 = [calc_dice(temp_class1_im, csf_atlas), calc_dice(temp_class2_im, csf_atlas), 
                                  calc_dice(temp_class3_im, csf_atlas)]
    dice2 = [calc_dice(temp_class1_im, wm_atlas), calc_dice(temp_class2_im, wm_atlas), 
                                  calc_dice(temp_class3_im, wm_atlas)]
    dice3 = [calc_dice(temp_class1_im, gm_atlas), calc_dice(temp_class2_im, gm_atlas), 
                                  calc_dice(temp_class3_im, gm_atlas)]
    csf_to_change = np.argmax(dice1) + 1
    wm_to_change = np.argmax(dice2) + 1
    gm_to_change = np.argmax(dice3) + 1
    
    
    #New y_pred
    y_pred_corrected_labels = np.zeros_like(y_pred)
    #Assign CSF to its correct label
    y_pred_corrected_labels[y_pred == csf_to_change] = CSF_label
    #Assign GM to its correct label
    y_pred_corrected_labels[y_pred == gm_to_change] = GM_label
    #Assign WM to its correct label
    y_pred_corrected_labels[y_pred == wm_to_change] = WM_label
    
    # Get weights back into original shape
    weight_csf_im = np.zeros(original_img_len)
    weight_gm_im = np.zeros(original_img_len)
    weight_wm_im = np.zeros(original_img_len)
    
    weight_csf_im[features_nonzero_row_indicies] = weights[:,csf_to_change-1]
    weight_gm_im[features_nonzero_row_indicies] = weights[:,gm_to_change-1]
    weight_wm_im[features_nonzero_row_indicies] = weights[:,wm_to_change-1]
    weight_csf_im = np.reshape(weight_csf_im, original_im.shape)
    weight_gm_im = np.reshape(weight_gm_im, original_im.shape)
    weight_wm_im = np.reshape(weight_wm_im, original_im.shape)
    
    # Multiply weights by each atlas
    csf_probs = weight_csf_im * csf_atlas
    gm_probs = weight_gm_im * gm_atlas
    wm_probs = weight_wm_im * wm_atlas
    
    # Assign GM, WM, CSF to voxel with highest probability
    GM = GM_label * np.nan_to_num((gm_probs > csf_probs) * (gm_probs > wm_probs))
    WM = WM_label * np.nan_to_num((wm_probs > csf_probs) * (wm_probs > gm_probs))
    CSF = CSF_label * np.nan_to_num((csf_probs > wm_probs) * (csf_probs > gm_probs))
    seg_im = GM + WM + CSF
    
    segment_im = np.zeros(original_img_len)
    segment_im = np.reshape(seg_im, original_im.shape)
    segment_nii = nib.Nifti1Image(segment_im, original_im.affine, original_im.header)

    return segment_nii


def seg_correct_labels_to_nii(original_im, y_pred, features_nonzero, features_nonzero_row_indicies, csf_atlas, 
                           gm_atlas, wm_atlas):
    """
    Transforms segmenation result to nii file, puts correct labels in place.
    The segmentation labels should be: 1) CSF (darkest) 2) GM (middle) 3) WM (light)
    
    Inputs: 
    original_im (nibabel): original image nii file
    y_pred (np array): labels for all non-zero points
    features_nonzero (np array): feature vector of only non-zero intensities
    features_nonzero_row_indicies (np array): indicies of non-zero points,
                                              same length as y_pred

    Returns:
    segment_nii (nibabel): segmented labels nii file        
    """
    
    # Create image with all 3 classes and random labels
    y_pred = y_pred + 1
    original_im_flat = original_im.get_data().copy().flatten()
    segment_im = np.zeros_like(original_im_flat)
    segment_im[features_nonzero_row_indicies] = y_pred
    segment_im = np.reshape(segment_im, original_im.shape)
    
    temp_class1_im = np.zeros_like(segment_im)
    temp_class2_im = np.zeros_like(segment_im)
    temp_class3_im = np.zeros_like(segment_im)
    
    #Assign class1 to 1
    temp_class1_im[segment_im == 1] = 1
    #Assign class2 to 2
    temp_class2_im[segment_im == 2] = 1
    #Assign class3 to 1
    temp_class3_im[segment_im == 3] = 1
    
    # Compute DICE between each class to determine which class it belongs to
    dice1 = [calc_dice(temp_class1_im, csf_atlas), calc_dice(temp_class2_im, csf_atlas), 
                                  calc_dice(temp_class3_im, csf_atlas)]
    dice2 = [calc_dice(temp_class1_im, wm_atlas), calc_dice(temp_class2_im, wm_atlas), 
                                  calc_dice(temp_class3_im, wm_atlas)]
    dice3 = [calc_dice(temp_class1_im, gm_atlas), calc_dice(temp_class2_im, gm_atlas), 
                                  calc_dice(temp_class3_im, gm_atlas)]
    csf_to_change = np.argmax(dice1) + 1
    wm_to_change = np.argmax(dice2) + 1
    gm_to_change = np.argmax(dice3) + 1
    
    
    #New y_pred
    y_pred_corrected_labels = np.zeros_like(y_pred)
    #Assign CSF to its correct label
    y_pred_corrected_labels[y_pred == csf_to_change] = CSF_label
    #Assign GM to its correct label
    y_pred_corrected_labels[y_pred == gm_to_change] = GM_label
    #Assign WM to its correct label
    y_pred_corrected_labels[y_pred == wm_to_change] = WM_label

    original_im_flat = original_im.get_data().copy().flatten()
    segment_im = np.zeros_like(original_im_flat)
    labels = np.copy(y_pred_corrected_labels)
    segment_im[features_nonzero_row_indicies] = labels
    segment_im = np.reshape(segment_im, original_im.shape)
    segment_nii = nib.Nifti1Image(segment_im, original_im.affine, original_im.header)

    return segment_nii

# Data Preparation

In [4]:
# put either 'mni' or 'our' (made from provided test images)
#use_atlas = 'mni'

MAX_STEPS = 30
min_change = 0.01
# select initial classes for each point, it can be either 'kmeans' or 'atlas'
#class_init = 'atlas' 

In [5]:
def gaussian_mixture(features, mean, cov):
    """
    Return Gaussian mixture function for a class.
    
    Inputs:
        features (numpy.ndarray): n by d dimentional array of features of points from feature space, where d is 
                                  is the dimentionality of feature space, n - number of points in the data.
        mean (numpy.ndarray): d-dimentional mean value.
        con (numpy.ndarray): d by d dimentional covariance matrix.
    
    Returns:
        (numpy.ndarray): Gaussian mixture for every point in feature space.
    """
    return np.exp(-0.5*(features - mean) * (1/cov) * np.transpose(features - mean)) / (2 * pi * sqrt(cov))


def membership_weight(p0, p1, p2, a0, a1, a2):
    """
    Return membership weights for three classes.
    
    Inputs:
        p0 (numpy.ndarray): mixture component for the 0th class, which is a probability distribution.
        p1 (numpy.ndarray): mixture component for the 1st class, which is a probability distribution.
        p2 (numpy.ndarray): mixture component for the 2nd class, which is a probability distribution.
        a0 (float): the probability of the 0th class.
        a1 (float): the probability for the 1st class.
        a2 (float): the probability for the 2nd class.
        
    Returns:
        (numpy.ndarray): membership weights for each point from the feature space, 
    """
    denominator = (p0 * a0) + (p1 * a1) + (p2 * a2)
    w0 = (p0 * a0) / denominator
    w1 = (p1 * a1) / denominator
    w2 = (p2 * a2) / denominator
    
    return np.stack((w0, w1, w2), axis=1)


def get_log_likelihood(class_dist, gauss_density):
    """
    Return loglikelihood.
    
    Parameters:
        class_dist: iterable with class distributions.
        gauss_density: iterable with Gaussian densities for each class.
    
    Returns:
        float: log likelihood value. 
    """
    for index, alpha in enumerate(class_dist):
        if index == 0:
            total_sum = alpha * gauss_density[index]
        else:
            total_sum += alpha * gauss_density[index]
    
    return np.sum(np.log(total_sum))


def save_dice(out_dice_path, img_names, dice_values):
    """
    Save DICE values into a csv file.
    Tissue type should be in this order: csf,gm,wm.
    
    Parameters:
        out_dice_path (str): output file path.
        img_names (list[str]): list of volumes used for dice calculation.
        dice_values (numpy.ndarray): array with dice coefficeints for each tissue type.
    
    Returns: None.
    """
    with open(out_dice_path, 'w+') as out_f:
        out_f.write('img,csf,gm,wm,\n')
        for index, row in enumerate(dice_values): 
            out_f.write(img_names[index] + ',' + ','.join(str(j) for j in row) + ',\n')

In [7]:
for use_atlas in ('our', 'mni'):   
    for class_init in ('kmeans','atlas'):
        if use_atlas == 'our':
            # Path to test image
            test_img_path = "./test-set/testing-images/"
            atlas_path = "./test-set/registration-results/our_atlases/"
            result_path = f"./test-set/segmentation-results/our-EM-atlas-segmentation_{class_init}-init/"
            gt_path = "./test-set/testing-labels/"
            mask_path = "./test-set/testing-mask/"

            out_dice_path = f'./dice-results/our_EM_atlas_{class_init}_dice.csv'

        elif use_atlas == 'mni':
            # Path to test image
            test_img_path = "./test-set/testing-images/"
            atlas_path = "./test-set/registration-results/mni_atlases/"
            result_path = f"./test-set/segmentation-results/mni-EM-atlas-segmentation_{class_init}-init/"
            gt_path = "./test-set/testing-labels/"
            mask_path = "./test-set/testing-mask/"

            out_dice_path = f'./dice-results/mni_EM_atlas_{class_init}_dice.csv'
        else:
            raise ValueError("our_atlas values should be either mni' or 'our'")

        onlydirs = [f[:-7] for f in listdir(test_img_path)]

        all_dice = np.zeros((len(onlydirs),3))

        for i, f in enumerate(onlydirs):
            # Load all data for EM algorithm
            test_data, test_img = read_im(join(test_img_path,f+'.nii.gz'))
            test_data = normalize(test_data)

            GM_atlas, _ = read_im(join(atlas_path, f,'gm','result.nii.gz'))
            WM_atlas, _ = read_im(join(atlas_path, f,'wm','result.nii.gz'))
            CSF_atlas, _ = read_im(join(atlas_path, f,'csf','result.nii.gz'))
            _, groundtruth_img = read_im(join(gt_path,f+"_3C.nii.gz"))
            mask_data, _ = read_im(join(mask_path,f+"_1C.nii.gz"))

            # Apply mask from GT image
            test_masked = apply_mask(test_data, mask_data)

            # Pre-process feature vector to remove background points from algorithm
            # and save those indicies to add back
            features = test_masked.copy().flatten()
            features = np.transpose(features)   
            features_nonzero_row_indicies = np.nonzero(features)
            features_nonzero = features[features_nonzero_row_indicies]

            if class_init == 'kmeans':
                features_nonzero_reshaped = features_nonzero.reshape(-1, 1)

                kmeans = KMeans(n_clusters=3, random_state=0, init='k-means++')\
                .fit(features_nonzero_reshaped)
                y_pred = kmeans.predict(features_nonzero_reshaped)
                centroids = kmeans.cluster_centers_

            elif class_init == 'atlas':
                GM_atlas_flat = GM_atlas.flatten()
                WM_atlas_flat = WM_atlas.flatten()
                CSF_atlas_flat = CSF_atlas.flatten()

                # row index shifted by +1 will correspond to tissue labels from ground-truth
                features_nonzero_pred = np.array((CSF_atlas_flat[features_nonzero_row_indicies],
                                                  WM_atlas_flat[features_nonzero_row_indicies],
                                                  GM_atlas_flat[features_nonzero_row_indicies]))
                y_pred = np.argmax(features_nonzero_pred, axis=0)

            else:
                raise ValueError("Incorrect class_init value, shoulb be one from ('kmeans', 'random')")

            # intialize EM algorithm
            class0 = features_nonzero[np.argwhere(y_pred == 0)[:,0]]
            class1 = features_nonzero[np.argwhere(y_pred == 1)[:,0]]
            class2 = features_nonzero[np.argwhere(y_pred == 2)[:,0]]

            # Compute mean and variance of each class
            mean0 = np.mean(class0, axis = 0)
            mean1 = np.mean(class1, axis = 0)
            mean2 = np.mean(class2, axis = 0)
            cov0 = np.cov(class0, rowvar = False)
            cov1 = np.cov(class1, rowvar = False)
            cov2 = np.cov(class2, rowvar = False)

            # Class distribution
            a0 = class0.shape[0] / features_nonzero.shape[0]
            a1 = class1.shape[0] / features_nonzero.shape[0]
            a2 = class2.shape[0] / features_nonzero.shape[0]

            # Compute Gaussian mixture model for each point
            p0 = gaussian_mixture(features_nonzero,  mean = mean0, cov = cov0)
            p1 = gaussian_mixture(features_nonzero,  mean = mean1, cov = cov1)
            p2 = gaussian_mixture(features_nonzero,  mean = mean2, cov = cov2)

            # # Compute membership weight for each point
            weights = membership_weight(p0, p1, p2, a0, a1, a2)
            # get initial log-likelihood
            log_likelihood = get_log_likelihood((a0, a1, a2), (p0, p1, p2))

            n_steps = 0

            while True:
                # Maximization step: Use that classification to reestimate the parameters
                # Class distribution
                counts = np.sum(weights, axis=0)

                a0 = counts[0] / len(features_nonzero)
                a1 = counts[1] / len(features_nonzero)
                a2 = counts[2] / len(features_nonzero)

                # Calculate mean and covariance for new classes
                mean0 = (1/counts[0]) * (weights[:, 0] @ features_nonzero)
                mean1 = (1/counts[1]) * (weights[:, 1] @ features_nonzero)
                mean2 = (1/counts[2]) * (weights[:, 2] @ features_nonzero)
                cov0 = (1/counts[0]) * ((weights[:, 0] * (features_nonzero - mean0)) @ (features_nonzero - mean0))
                cov1 = (1/counts[1]) * ((weights[:, 1] * (features_nonzero - mean1)) @ (features_nonzero - mean1))
                cov2 = (1/counts[2]) * ((weights[:, 2] * (features_nonzero - mean2)) @ (features_nonzero - mean2))

                p0 = gaussian_mixture(features_nonzero,  mean = mean0, cov = cov0)
                p1 = gaussian_mixture(features_nonzero,  mean = mean1, cov = cov1)
                p2 = gaussian_mixture(features_nonzero,  mean = mean2, cov = cov2)

                # Compute membership weight for each point
                weights = membership_weight(p0, p1, p2, a0, a1, a2)

                log_likelihood_new = get_log_likelihood((a0, a1, a2), (p0, p1, p2))

                dist_change = abs((log_likelihood_new - log_likelihood) / log_likelihood)
                print("********************************************************************")
                print(f"Img {f}")
                print("Step %d" % n_steps)
                print("Distribution change %f" % dist_change)
                print(a0, a1, a2)

                n_steps += 1

                # check whether we reached desired precision or max number of steps
                if (n_steps >= MAX_STEPS) or (dist_change <= min_change):
                    print("Loop stopped")
                    break
                else:
                    log_likelihood = log_likelihood_new

            y_pred = np.argmax(weights, axis=1)
            segment_nii_atlas = integrate_atlas_nii(test_img, y_pred, features_nonzero, 
                                       features_nonzero_row_indicies, weights, CSF_atlas, 
                                       GM_atlas, WM_atlas)

            # Calculate DICE
            all_dice[i,0], all_dice[i,1], all_dice[i,2] = dice_similarity(segment_nii_atlas, groundtruth_img)

            # Make directory to save result seg
            new_dir = join(result_path,f)
            os.mkdir(new_dir)
            nib.save(segment_nii_atlas, join(new_dir,'atlas_EM_seg.nii.gz'))

        save_dice(out_dice_path, onlydirs, all_dice)
        print(np.mean(all_dice, axis=0))

********************************************************************
Img 1113
Step 0
Distribution change 0.050032
0.32737014268516507 0.31531050798286986 0.3569843944267925
********************************************************************
Img 1113
Step 1
Distribution change 0.014069
0.3263680162222968 0.31946500463776123 0.35390482228493614
********************************************************************
Img 1113
Step 2
Distribution change 0.006483
0.3266946330248071 0.32266094018274755 0.35028492628919894
Loop stopped
********************************************************************
Img 1110
Step 0
Distribution change 0.040015
0.32775630807415385 0.3504314080001712 0.3212619656108016
********************************************************************
Img 1110
Step 1
Distribution change 0.011843
0.3343348994650039 0.3492373671870692 0.3160420296194129
********************************************************************
Img 1110
Step 2
Distribution change 0.005683
0.339104273

********************************************************************
Img 1004
Step 0
Distribution change 0.037770
0.35404707194246066 0.2985767312086525 0.347100230156095
********************************************************************
Img 1004
Step 1
Distribution change 0.011268
0.3518798490173312 0.29474265593438914 0.3531718143820003
********************************************************************
Img 1004
Step 2
Distribution change 0.005190
0.34908448963638244 0.2927228941917623 0.35786136738484925
Loop stopped
********************************************************************
Img 1116
Step 0
Distribution change 0.050462
0.2433830718981443 0.3832801609358007 0.37294261470101264
********************************************************************
Img 1116
Step 1
Distribution change 0.014520
0.24533572051662444 0.38442365592709177 0.36989227097100946
********************************************************************
Img 1116
Step 2
Distribution change 0.006740
0.246584445

********************************************************************
Img 1005
Step 7
Distribution change 0.022906
0.030859059511494213 0.447417333207153 0.5216982045306452
********************************************************************
Img 1005
Step 8
Distribution change 0.012116
0.030361064573577835 0.4389006560896709 0.5307599374251406
********************************************************************
Img 1005
Step 9
Distribution change 0.009208
0.030032245408564275 0.4297095221135883 0.5402949369411282
Loop stopped
********************************************************************
Img 1025
Step 0
Distribution change 0.036449
0.035436366078676715 0.44262002194452826 0.5220804451362066
********************************************************************
Img 1025
Step 1
Distribution change 0.015060
0.035154756923371486 0.4485132864721804 0.5158723815465311
********************************************************************
Img 1025
Step 2
Distribution change 0.021357
0.033868

********************************************************************
Img 1128
Step 1
Distribution change 0.015775
0.03369385886730183 0.36765383256195133 0.5990762293868848
********************************************************************
Img 1128
Step 2
Distribution change 0.004879
0.03527097487362055 0.3686761192421965 0.5958446934725681
Loop stopped
********************************************************************
Img 1023
Step 0
Distribution change 0.026490
0.011152225870129021 0.41967875662172544 0.5692760319167559
********************************************************************
Img 1023
Step 1
Distribution change 0.028756
0.010960238309421657 0.421769360907888 0.567188483204986
********************************************************************
Img 1023
Step 2
Distribution change 0.038782
0.011303914963315365 0.421420236891783 0.566635769224097
********************************************************************
Img 1023
Step 3
Distribution change 0.046677
0.0120302553

********************************************************************
Img 1038
Step 6
Distribution change 0.011464
0.012288710570236854 0.4221754460158974 0.5653027465061683
********************************************************************
Img 1038
Step 7
Distribution change 0.007030
0.01229942945330551 0.41383223714705214 0.5736616543311682
Loop stopped
********************************************************************
Img 1018
Step 0
Distribution change 0.032364
0.012104863503246178 0.4372621043488579 0.5504448389649573
********************************************************************
Img 1018
Step 1
Distribution change 0.038436
0.0116000501244092 0.4402922269279663 0.5484314101439905
********************************************************************
Img 1018
Step 2
Distribution change 0.047584
0.011737936727336739 0.440359083104081 0.5476970828458176
********************************************************************
Img 1018
Step 3
Distribution change 0.053564
0.012216459

********************************************************************
Img 1107
Step 2
Distribution change 0.082552
0.015845281716190558 0.4222492256303178 0.5616806709506331
********************************************************************
Img 1107
Step 3
Distribution change 0.066877
0.016789458622717815 0.4204723359500329 0.561478666410817
********************************************************************
Img 1107
Step 4
Distribution change 0.044446
0.01754217988646367 0.4163408201952703 0.5645310152564014
********************************************************************
Img 1107
Step 5
Distribution change 0.024509
0.01799589791245303 0.41073377324927884 0.5698513396640973
********************************************************************
Img 1107
Step 6
Distribution change 0.012535
0.018237318275488267 0.40425490676643594 0.5760540840157002
********************************************************************
Img 1107
Step 7
Distribution change 0.012130
0.018410605090559876 0

********************************************************************
Img 1019
Step 1
Distribution change 0.009024
0.3394953393176299 0.33265636404953886 0.32757074055882995
Loop stopped
********************************************************************
Img 1038
Step 0
Distribution change 0.041955
0.36430078890979384 0.33625974935526676 0.29920339799353546
********************************************************************
Img 1038
Step 1
Distribution change 0.012383
0.36240019436907966 0.3414808800509253 0.29598944367976027
********************************************************************
Img 1038
Step 2
Distribution change 0.005631
0.35989263790139664 0.34541836586944297 0.29447301832269596
Loop stopped
********************************************************************
Img 1018
Step 0
Distribution change 0.045168
0.3351030043180583 0.3594777518351093 0.30515912642426557
********************************************************************
Img 1018
Step 1
Distribution change 0.0

********************************************************************
Img 1025
Step 3
Distribution change 0.060733
0.1427684067149126 0.28571540857730077 0.5720538799635718
********************************************************************
Img 1025
Step 4
Distribution change 0.037076
0.1421153057432349 0.2874288754595033 0.5728600199097491
********************************************************************
Img 1025
Step 5
Distribution change 0.011567
0.14116641552804993 0.287416312640995 0.5719369508127459
********************************************************************
Img 1025
Step 6
Distribution change 0.004933
0.1420652310993389 0.2857688833513378 0.571619855558939
Loop stopped
********************************************************************
Img 1122
Step 0
Distribution change 0.304114
0.15803258469058112 0.2948766133763567 0.5465534866492405
********************************************************************
Img 1122
Step 1
Distribution change 0.122016
0.154250752153815

********************************************************************
Img 1018
Step 1
Distribution change 0.068099
0.11525683023635458 0.2942630420803461 0.5906312357289876
********************************************************************
Img 1018
Step 2
Distribution change 0.043209
0.11173596096646328 0.3044983627326006 0.5859142762039129
********************************************************************
Img 1018
Step 3
Distribution change 0.010960
0.10884119351471097 0.311470954754183 0.5813928137354214
********************************************************************
Img 1018
Step 4
Distribution change 0.001148
0.10746098974842883 0.3146817144322962 0.5790058410267015
Loop stopped
********************************************************************
Img 1004
Step 0
Distribution change 0.060790
0.12239961777588132 0.29258424517794746 0.5823749272203622
********************************************************************
Img 1004
Step 1
Distribution change 0.063086
0.12011093696